In [1]:
import json
import datetime
import pandas as pd
import ipaddress
import random
from pymongo import MongoClient
from random import randint

In [11]:
datetime.datetime.now().strftime("%H:%M:%S - %m %d %Y")

'12:18:54 - 05 15 2020'

In [12]:
datetime.datetime.now().strftime("%Y%m%d%H%M%S")

'20200515121855'

In [13]:
datetime.datetime.now().replace(microsecond  = 0)

datetime.datetime(2020, 5, 15, 12, 18, 55)

In [8]:
# Read 1000000 records into MongoDB database
# Get the process time
client = MongoClient(port=27017)
db = client.netflow

start = datetime.datetime.now()
with open('data/20200515134659.json') as record:
    new_record = json.load(record)

db.testlist.insert_many(new_record)
client.close()

end = datetime.datetime.now()
print(end-start)

#print(new_record)

0:00:22.087487


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
client = MongoClient(port=27017)
db=client.netflow
# Count the total flows in whole database
start = datetime.datetime.now()
total_flows = db.testlist.aggregate([
    {
        '$group':{
            '_id': "null",
            'total': { '$sum': "$flows"}
        }
    }
])
for total in total_flows:
    print(total)
    
client.close()    
end = datetime.datetime.now()
print(end-start)

{'_id': 'null', 'total': 30000722}
0:00:01.269110


In [20]:
client = MongoClient(port=27017)
db=client.netflow
# Search all flows with UDP in 20200515 13:36:42
start = datetime.datetime.now()
udp_flows = db.testlist.aggregate([
    {   '$match': {'timestamp': "20200515134632"}},
    {   '$group': {
            '_id': "$protocol",
            'total': { '$sum': "$flows"}
        }
    }
])
for udp_flow in udp_flows:
    print(udp_flow)
end = datetime.datetime.now()
print(end-start)

{'_id': 'TCP', 'total': 4390}
{'_id': 'UDP', 'total': 3958}
0:00:00.706507


In [7]:
int(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))

20200520065322

In [6]:
datetime.datetime.now().strftime("%Y%m%d%H%M%S")

'20200520065310'

In [11]:
# generate 10 X 1000000 records with "sourceIP","destinationIP","timestamp"
# get the process time
start = datetime.datetime.now()
MAX_IP_INT = 4200000000
N=1000000

protocols = ['TCP','UDP']

for i in range(10):
    netflow = []
    now_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i)
    for n in range(N):
        IPs = {
        "srcaddr" :  "{}".format(str(ipaddress.ip_address(randint(0, MAX_IP_INT)))),
        "srcport" :  randint(0,65535),
        "dstaddr" :  "{}".format(str(ipaddress.ip_address(randint(0, MAX_IP_INT)))),        
        "dstport" :  randint(0,65535),
        "first"   :  int(now_time.strftime("%Y%m%d%H%M%S")),
        "last"    :  int(now_time.strftime("%Y%m%d%H%M%S")),
        "flows"   :  randint(0,30),
        "packets" :  randint(0,500),
        "bytes"   :  randint(0,1000000),
        "protocol":  random.choice(protocols),
        }
        netflow.append(IPs)

    json_object = json.dumps(netflow, indent = 3)
    now_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i)
    file_name = now_time.strftime("%Y%m%d%H%M%S") + '.json'
    with open(file_name, "w") as outfile:
        outfile.write(json_object)
        outfile.close()
    
end = datetime.datetime.now()
print(end-start)

0:05:24.611934


In [14]:
# 1. Read 1000000 records into MongoDB database
#    Get the process time

client = MongoClient(port=27017)
db = client.netflow

start = datetime.datetime.now()
with open('data/20200520074458.json') as record:
    new_record = json.load(record)

db.testlist.insert_many(new_record)


client.close()

end = datetime.datetime.now()
print(end-start)


0:00:33.311647


In [7]:
# 2 Sort the database by the data transfer for the whole database and get the top 10 records
client = MongoClient(port=27017)
db = client.netflow

start = datetime.datetime.now()
top_transfers = db.testlist.aggregate([{'$sort': {"bytes": -1}},{'$limit':10},{'$project':{"_id":0}}]);

for top_transfer in top_transfers:
    print(top_transfer)
    
client.close()

end = datetime.datetime.now()
print(end-start)

{'srcaddr': '157.64.227.78', 'srcport': 7988, 'dstaddr': '212.75.236.165', 'dstport': 1827, 'first': 20200520065855, 'last': 20200520065855, 'flows': 28, 'packets': 377, 'bytes': 1000000, 'protocol': 'TCP'}
{'srcaddr': '175.44.213.207', 'srcport': 54042, 'dstaddr': '172.7.9.195', 'dstport': 32825, 'first': 20200520065525, 'last': 20200520065525, 'flows': 10, 'packets': 57, 'bytes': 1000000, 'protocol': 'UDP'}
{'srcaddr': '40.81.184.53', 'srcport': 51799, 'dstaddr': '153.81.36.246', 'dstport': 8966, 'first': 20200520065859, 'last': 20200520065859, 'flows': 2, 'packets': 416, 'bytes': 1000000, 'protocol': 'UDP'}
{'srcaddr': '102.32.97.120', 'srcport': 10545, 'dstaddr': '36.129.28.217', 'dstport': 2738, 'first': 20200520065716, 'last': 20200520065716, 'flows': 8, 'packets': 148, 'bytes': 1000000, 'protocol': 'TCP'}
{'srcaddr': '185.79.124.30', 'srcport': 37089, 'dstaddr': '219.72.121.151', 'dstport': 33571, 'first': 20200520065811, 'last': 20200520065811, 'flows': 15, 'packets': 306, 'byt

In [8]:
# 2 Sort the database by the data transfer for the last 10 minutes and get the top 10 records
client = MongoClient(port=27017)
db = client.netflow

start = datetime.datetime.now()
top_transfers = db.testlist.aggregate([
    {'$match': {'first': {'$gt': 20200520065700, '$lte':20200520070255}}},
    {'$sort': {"bytes": -1}},
    {'$limit':10},
    {'$project':{"_id":0}}
    ]);

for top_transfer in top_transfers:
    print(top_transfer)
    
client.close()

end = datetime.datetime.now()
print(end-start)

{'srcaddr': '40.81.184.53', 'srcport': 51799, 'dstaddr': '153.81.36.246', 'dstport': 8966, 'first': 20200520065859, 'last': 20200520065859, 'flows': 2, 'packets': 416, 'bytes': 1000000, 'protocol': 'UDP'}
{'srcaddr': '157.64.227.78', 'srcport': 7988, 'dstaddr': '212.75.236.165', 'dstport': 1827, 'first': 20200520065855, 'last': 20200520065855, 'flows': 28, 'packets': 377, 'bytes': 1000000, 'protocol': 'TCP'}
{'srcaddr': '102.32.97.120', 'srcport': 10545, 'dstaddr': '36.129.28.217', 'dstport': 2738, 'first': 20200520065716, 'last': 20200520065716, 'flows': 8, 'packets': 148, 'bytes': 1000000, 'protocol': 'TCP'}
{'srcaddr': '4.158.159.166', 'srcport': 2012, 'dstaddr': '15.167.10.244', 'dstport': 20292, 'first': 20200520065711, 'last': 20200520065711, 'flows': 9, 'packets': 59, 'bytes': 999999, 'protocol': 'UDP'}
{'srcaddr': '185.79.124.30', 'srcport': 37089, 'dstaddr': '219.72.121.151', 'dstport': 33571, 'first': 20200520065811, 'last': 20200520065811, 'flows': 15, 'packets': 306, 'bytes

In [9]:
client = MongoClient(port=27017)
db=client.netflow
# 3 Count the total flows in whole database
start = datetime.datetime.now()
total_flows = db.testlist.aggregate([
    {
        '$group':{
            '_id': "null",
            'total': { '$sum': "$flows"}
        }
    }
])
for total in total_flows:
    print(total)
    
client.close()    
end = datetime.datetime.now()
print(end-start)

{'_id': 'null', 'total': 74983073}
0:00:03.699538


In [10]:
client = MongoClient(port=27017)
db=client.netflow
# Count the total flows for the last 10 minutes
start = datetime.datetime.now()
total_flows = db.testlist.aggregate([
    {'$match': {'first': {'$gt': 20200520065700, '$lte':20200520070255}}},
    {
        '$group':{
            '_id': "null",
            'total': { '$sum': "$flows"}
        }
    }
])
for total in total_flows:
    print(total)
    
client.close()    
end = datetime.datetime.now()
print(end-start)

{'_id': 'null', 'total': 59972470}
0:00:05.177900


In [ ]:
client = MongoClient(port=27017)
db=client.netflow
# Search all flows with UDP for the last 10 minutes
start = datetime.datetime.now()
udp_flows = db.testlist.aggregate([
    {   '$match': {'first': {'$gt': 20200520065700, '$lte':20200520070255}}},
    {   '$group': {
            '_id': "$protocol",
            'total': { '$sum': "$bytes"}
        }
    }
])
for udp_flow in udp_flows:
    print(udp_flow)
end = datetime.datetime.now()
print(end-start)

In [ ]:
# plot every 5 minutes each TCP/UDP traffic

In [ ]:
# plot every 5 minutes total traffic in the network

In [12]:
from pymongo import MongoClient
from random import randint
import datetime
start = datetime.datetime.now()
# step 1: Connect to MongoDB - Note: Change connection string as needed
client = MongoClient(port=27017)
db = client.netflow

db.testlist.insert_many(netflow)